<a href="https://colab.research.google.com/github/SamoaChen/Natural_Language_Class_Projects/blob/main/Zhiyang_HW1_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-----IMPORT PACKAGES
import numpy as np
import math
import nltk 
import random
import string
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
lancaster=LancasterStemmer()
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [ ]:
#-----TRAINING AND TESTING FUNCTIONS
#CREATE A CLASS FOR MODEL PARAMETERS
class TrainParameters:
    def __init__(self,logprior, loglikelihood, classes, V, weight):
        self.logprior = logprior
        self.loglikelihood = loglikelihood
        self.classes = classes
        self.weight = weight
        self.V = V
        
#CREATE THE TRAINING FUNCTION
def naive_bayes_train(file_name, stop_word_num, smoothe, coef):
    class_dict = dict()

    #INITIATE LOCATION FOR STORING DOC NUMBER
    class_dict['N_doc'] = 0 
    
    #INITIATE LOCATION FOR STORING ALL WORDS
    V = dict()
    
    #CALCULATE STOP WORDS
    stop_words = find_stop_words(file_name, stop_word_num)

    #READ FILE
    train_file = open(file_name,'r')

    #ITERATE EVERY DOCUMENT
    for line in train_file:
        train_doc_info = line.split()
        #POPUATE CLASS_DICT DICTIONARIES
        if train_doc_info[1] in class_dict.keys():
            class_dict[train_doc_info[1]]['N_class'] += 1
        else:
            class_dict[train_doc_info[1]] = dict()
            class_dict[train_doc_info[1]]['N_class'] = 1
            class_dict[train_doc_info[1]]['N_word'] =0
        #COUNT DOCUMENT NUMBER
        class_dict['N_doc'] += 1
        
        #LOOP WORDS FROM EACH CLASS
        text = open(train_doc_info[0]).read()
        text_tokenized = tknzr.tokenize(text)
        #REPLACE WORD WITH POS (doesn't work)
        #text_tokenized = [t for w, t in nltk.pos_tag(text_tokenized)]
        
        for word in text_tokenized:
            #word = lancaster.stem(word) #doesn't workd
            #word = lemmatizer.lemmatize(word) #(doesn't work)
            #word = ps.stem(word) #(doesn't work)
            #if word in string.punctuation:   #doesn't work
                #continue
            if word in stop_words:
                continue
            elif word not in class_dict[train_doc_info[1]].keys():
                class_dict[train_doc_info[1]][word] = 1
                class_dict[train_doc_info[1]]['N_word'] += 1
            else:
                class_dict[train_doc_info[1]][word] += 1
                class_dict[train_doc_info[1]]['N_word'] += 1
            #TABULATE WORD TO V DICTIONARY
            if word not in V.keys():
                V[word] = 1
            else:
                V[word] += 1
    
    #CLOSE TRAIN_FILE OBJECT
    train_file.close()
    
    #CREATE LIST OF CLASSES
    classes = list()
    for i in class_dict.keys():
        classes.append(i)
    classes.remove('N_doc')
    
    #SELECT THE SMOOTHING METHOD
    if smoothe == 'lap':
        return loglikelihood_laplace(class_dict, V, classes, coef)
    elif smoothe == 'JM':
        return loglikelihood_jelinek_mercer(class_dict, V, classes, coef)
    elif smoothe == 'dir':
        return loglikelihood_dirichlet(class_dict, V, classes, coef)

    
#CREATE TRAINING FUNCTION FOR THE BAGGING APPROACH
def naive_bayes_bagging_train(file_name, bags, portion_percent): 
    #INITIATE A LIST TO STORE ALL TRAINED MODEL OBJECT
    bag_list = []
    #CREATE A NUMBER ARRAY TO SELECT FROM
    line_count = sum(1 for line in open(file_name))
    number_list = np.arange(1, line_count+1,1)
    number_in_bag = int(line_count*portion_percent)
    #TRAINING FOR EACH BAG
    for bag in range(bags):
        partition_list = np.random.choice(number_list, number_in_bag, replace = True)
        write_train_bag(partition_list, file_name)
        logprior, loglikelihood, classes, V = naive_bayes_train('bag.labels', 25, 'lap', 0.094)
        naive_bayes_test_validation('test.labels', logprior, loglikelihood, classes, V)
        right, wrong, weight = compute_accuracy('test.labels','predicted_for_bagging.labels')
        bag_list.append(TrainParameters(logprior, loglikelihood, classes, V, weight*(right+wrong)))
    
    return bag_list

#CREATE TESTING FUNCTION FOR THE BAGGING APPROACH
def naive_bayes_bagging_test(test_file_name, bag_list):
    #CREATE OUTPUT FILE
    output_file = open('predicted.labels', 'w+')
    
    #READ TEST FILE
    test_file = open(test_file_name, 'r')
    
    #ITERATE THROUGH EVERY DOCUMENT
    for line in test_file:
        #INITIATE DICTIONARY TO STORE CLASS COUNT GIVEN BY EACH MODEL
        class_count_bagging = dict()
        test_doc_info = line.split()
        
        #TOKENIZE WORDS
        text = open(test_doc_info[0]).read()
        text_tokenized = tknzr.tokenize(text)
        
        #ITERATE THROUGH ALL MODELS
        for bag in range(len(bag_list)):
            #INITIATE DICTIONARY TO STORE CLASS PROBABILITIES
            class_prob = dict()
            
            for class_type in bag_list[bag].classes:
                class_prob[class_type] = bag_list[bag].logprior[class_type]
                for word in text_tokenized:
                    if word in bag_list[bag].V.keys():
                        class_prob[class_type] += bag_list[bag].loglikelihood[class_type][word]
            
            max_class = sorted(class_prob, key=class_prob.get, reverse=True)[0]
            
            if max_class not in class_count_bagging.keys():
                class_count_bagging[max_class] = 1*bag_list[bag].weight
            else:
                class_count_bagging[max_class] += 1*bag_list[bag].weight
        
        #FIND THE CLASS THAT IS THE MOST CLASSIFIED
        max_class_bagging = sorted(class_count_bagging, key=class_count_bagging.get, reverse=True)[0]
        
        #WRITE TO DOCUMENT
        output_file.write(test_doc_info[0]+' '+max_class+'\n')
    
    output_file.close()
    test_file.close()

#CREATE TESTING FUNCTION
def naive_bayes_test_validation(test_file_name, logprior, loglikelihood, classes, V):  
    #CREATE OUTPUT FILE
    output_file = open('predicted_for_bagging.labels', 'w+')
    
    #READ TEST FILE
    test_file = open(test_file_name, 'r')
    
    #ITERATE EVERY DOCUMENT
    for line in test_file:
        #INITIATE DICTIONARY TO STORE CLASS PROBABILITIES
        class_prob = dict()
        test_doc_info = line.split()  
        
        #TOKENIZE WORDS
        text = open(test_doc_info[0]).read()
        text_tokenized = tknzr.tokenize(text)
        
        for class_type in classes:
            class_prob[class_type] = logprior[class_type]
            for word in text_tokenized:
                if word in V.keys():
                    class_prob[class_type] += loglikelihood[class_type][word]
        
        #FIND THE CLASS THAT OUTPUTS LARGEST PROBABILITY
        max_class = sorted(class_prob, key=class_prob.get, reverse=True)[0]
        
        #WRITE TO DOCUMENT
        output_file.write(test_doc_info[0]+' '+max_class+'\n')
    
    output_file.close()
    test_file.close()
    
#CREATE TESTING FUNCTION
def naive_bayes_test(test_file_name, logprior, loglikelihood, classes, V):  
    #CREATE OUTPUT FILE
    output_file = open('predicted.labels', 'w+')
    
    #READ TEST FILE
    test_file = open(test_file_name, 'r')
    
    #ITERATE EVERY DOCUMENT
    for line in test_file:
        #INITIATE DICTIONARY TO STORE CLASS PROBABILITIES
        class_prob = dict()
        test_doc_info = line.split()  
        
        #TOKENIZE WORDS
        text = open(test_doc_info[0]).read()
        text_tokenized = tknzr.tokenize(text)
        
        for class_type in classes:
            class_prob[class_type] = logprior[class_type]
            for word in text_tokenized:
                if word in V.keys():
                    class_prob[class_type] += loglikelihood[class_type][word]
        
        #FIND THE CLASS THAT OUTPUTS LARGEST PROBABILITY
        max_class = sorted(class_prob, key=class_prob.get, reverse=True)[0]
        
        #WRITE TO DOCUMENT
        output_file.write(test_doc_info[0]+' '+max_class+'\n')
    
    output_file.close()
    test_file.close()


#CREATE FUNCTION FOR FINDING STOP WORDS
def find_stop_words(file_name, stop_word_num):
    #INITIATE LOCATION FOR STORING ALL WORDS
    V = dict()
    
    #READ FILE
    train_file = open(file_name,'r') 
    
    #ITERATE EVERY DOCUMENT
    for line in train_file:    
        train_doc_info = line.split()
        
        #LOOP WORDS FROM EACH CLASS
        text = open(train_doc_info[0]).read()
        text_tokenized = tknzr.tokenize(text)
        
        for word in text_tokenized:        
            if word in string.punctuation:
                continue        
        
            #TABULATE WORD TO V DICTIONARY
            if word not in V.keys():
                V[word] = 1
            else:
                V[word] += 1
    
    #CLOSE TRAIN_FILE OBJECT
    train_file.close()
    
    #FIND 10 STOP WORDS 
    stop_words = sorted(V, key=V.get, reverse=True)[:stop_word_num]        
    
    return stop_words


        
#CREATE FUNCTION FOR LAPACE SMOOTHING LOG LIKELIHOOD
def loglikelihood_laplace(class_dict, V, classes, coef):
    #SMOOTHING FACTOR
    lap = coef
    #CREATE LOGPRIOR AND LOGLIKELIHOOD DICTIONARIES
    logprior = dict()
    loglikelihood = dict()
    #CALCULATE MAGNITUDE OF V
    V_length = len(list(V))
    
    #CALCULATE THE LOGPRIOR
    for class_type in classes:
        logprior[class_type] = math.log(class_dict[class_type]['N_class']/class_dict['N_doc'])
    
    #CALCULATE THE LOGLIKELIHOOD
    for word in V:
        for class_type in classes:
            if word in class_dict[class_type].keys():
                word_loglikelihood = math.log((class_dict[class_type][word]+lap)/(class_dict[class_type]['N_word']+V_length*lap))
            else:
                word_loglikelihood = math.log(lap/(class_dict[class_type]['N_word']+V_length*lap))
            
            if class_type not in loglikelihood.keys():
                loglikelihood[class_type] = dict()
                loglikelihood[class_type][word] = word_loglikelihood
            else:
                loglikelihood[class_type][word] = word_loglikelihood
                

    return logprior, loglikelihood, classes, V

#CREATE FUNCTION FOR JELINEK-MERCER SMOOTHING LOG LIKELIHOOD
def loglikelihood_jelinek_mercer(class_dict, V, classes, coef):
    #COEFFICIENT OF INTERPOLATION
    beta = coef
    
    #CREATE LOGPRIOR AND LOGLIKELIHOOD DICTIONARIES
    logprior = dict()
    loglikelihood = dict()
    #CALCULATE MAGNITUDE OF V
    V_length = len(list(V))
    
    #CALCULATE THE LOGPRIOR
    for class_type in classes:
        logprior[class_type] = math.log(class_dict[class_type]['N_class']/class_dict['N_doc'])
    
    #CALCULATE THE LOGLIKELIHOOD
    for word in V:
        #INITIATE PLACE TO STORE NUMERATOR AND DENOMINATOR
        numerator = 0
        denominator = 0
        #BUILDING THE COLLECTION LM, P_w_c
        for class_type in classes:
            if word in class_dict[class_type].keys():
                numerator += class_dict[class_type][word]
            denominator += class_dict[class_type]['N_word']
        P_w_c = numerator/denominator
        
        #CALCULATE LOGLIKELIHOOD FOR EACH WORD
        for class_type in classes:
            if word in class_dict[class_type].keys():
                word_loglikelihood = math.log((1-beta)*(class_dict[class_type][word]/class_dict[class_type]['N_word'])+beta*P_w_c)
            else:
                word_loglikelihood = math.log(beta*P_w_c)
                
            if class_type not in loglikelihood.keys():
                loglikelihood[class_type] = dict()
                loglikelihood[class_type][word] = word_loglikelihood
            else:
                loglikelihood[class_type][word] = word_loglikelihood 
    
    return logprior, loglikelihood, classes, V
    
#CREATE FUNCTION FOR DIRICHLET PRIOR SMOOTHING LOG LIKELIHOOD
def loglikelihood_dirichlet(class_dict, V, classes, coef):
    #COEFFICIENT OF INTERPOLATION
    mu = coef

    #CREATE LOGPRIOR AND LOGLIKELIHOOD DICTIONARIES
    logprior = dict()
    loglikelihood = dict()
    #CALCULATE MAGNITUDE OF V
    V_length = len(list(V))
    
    #CALCULATE THE LOGPRIOR
    for class_type in classes:
        logprior[class_type] = math.log(class_dict[class_type]['N_class']/class_dict['N_doc'])
        
    #CALCULATE THE LOGLIKELIHOOD
    for word in V:
        #INITIATE PLACE TO STORE NUMERATOR AND DENOMINATOR
        numerator = 0
        denominator = 0
        #BUILDING THE COLLECTION LM, P_w_c
        for class_type in classes:
            if word in class_dict[class_type].keys():
                numerator += class_dict[class_type][word]
            denominator += class_dict[class_type]['N_word']
        P_w_c = numerator/denominator
        
        #CALCULATE LOGLIKELIHOOD FOR EACH WORD
        for class_type in classes:
            if word in class_dict[class_type].keys():
                word_loglikelihood = math.log((class_dict[class_type]['N_word']/(class_dict[class_type]['N_word']+mu))*(class_dict[class_type][word]/class_dict[class_type]['N_word'])+(mu/(mu+class_dict[class_type]['N_word']))*P_w_c)
                #word_loglikelihood = math.log((class_dict[class_type]['N_word']+mu*P_w_c)/(class_dict[class_type]['N_word']+mu))
            else:
                word_loglikelihood = math.log((mu/(mu+class_dict[class_type]['N_word']))*P_w_c)
                #word_loglikelihood = math.log(mu*P_w_c/(class_dict[class_type]['N_word']+mu))

                
            if class_type not in loglikelihood.keys():
                loglikelihood[class_type] = dict()
                loglikelihood[class_type][word] = word_loglikelihood
            else:
                loglikelihood[class_type][word] = word_loglikelihood 
    
    return logprior, loglikelihood, classes, V
  
#COMPUTE ACCURACY FUNCTION
def compute_accuracy(actual_label_file,pred_label_file):
    #INITIATE CORRECT AND INCORRECT VARIABLE
    correct = 0
    incorrect = 0
    #COMPARING TWO FILES
    actual_label = open(actual_label_file,'r')
    pred_label = open(pred_label_file,'r')
    for line_actual in actual_label:
        line_actual_split = line_actual.split()
        #READ ONE LINE FROM PREDICTED FILE
        line_pred = pred_label.readline()
        line_pred_split = line_pred.split()
        #COMPARING
        if line_pred_split[1] == line_actual_split[1]:
            correct+=1
        else:
            incorrect+=1
    #CLOSING FILES
    actual_label.close()
    pred_label.close()
    #CALCULATE ACCURACY
    accuracy = correct/(correct+incorrect)
    
    return correct, incorrect, accuracy

#WRITE TRAIN FILE FOR THE BAGGING APPROACH
def write_train_bag(partition_list, file_name):
    line_current = 1
    bag = open('bag.labels','w+')
    test = open('test.labels','w+')
    file = open(file_name,'r')
    for line in file:
        if line_current in partition_list:
            iteration = np.sum(partition_list == line_current)
            for ite in range(iteration):
                bag.write(line)
        else:
            test.write(line)
        line_current += 1
    file.close()
    bag.close()
    test.close()

#WRITE TRAIN AND TEST FILES
def write_train_test(test_array, file_name):
    line_current = 1
    train = open('k_fold_folder/train.labels','w+')
    test = open('k_fold_folder/test.labels','w+')
    file = open(file_name,'r')
    #ITERATE AND WRITE THE TRAIN AND TEST FILE
    for line in file:
        if line_current in test_array:
            test.write(line)
            line_current += 1
        else:
            train.write(line)
            line_current += 1
    file.close()
    test.close()
    train.close()
         
    
#K FOLD CROSS VALIDATION
def k_fold_cross_validation(file_name, K, stop_word_num, smoothe, coef):
    #INITATE VARIABLE FOR STORING COREECT AND INCORRECT NUMBER
    correct_sum = 0
    incorrect_sum = 0
    #COUNT LINE NUMBER IN THE FILE
    line_count = sum(1 for line in open(file_name))
    number_list = np.arange(1,line_count+1,1)
    random.shuffle(number_list)
    #PARTITION SET INTO K PARTS
    partition_list = np.array_split(number_list, K)
    #ITERATE K POSSIBLE TRAINS AND TESTS
    for list in range(K):
        write_train_test(partition_list[list], file_name)
        logprior, loglikelihood, classes, V = naive_bayes_train('k_fold_folder/train.labels', stop_word_num, smoothe, coef) 
        naive_bayes_test('k_fold_folder/test.labels', logprior, loglikelihood, classes, V)
        correct, incorrect, _= compute_accuracy('k_fold_folder/test.labels','predicted.labels')
        correct_sum += correct
        incorrect_sum += incorrect
    accuracy = correct_sum/(correct_sum+incorrect_sum)
    return accuracy

    
#logprior, loglikelihood, classes, V = naive_bayes_train("train.labels") 
#naive_bayes_test('test.labels', logprior, loglikelihood, classes, V)

#k_fold_cross_validation('corpus2_train.labels',3,50,'lap', 0.001)

### Sample function call for bagging with naive bayes

In [ ]:
bag_list = naive_bayes_bagging_train('corpus1_train.labels', 10, 0.6)
naive_bayes_bagging_test('corpus1_test.list', bag_list)

### Grid search to find the best parameters for bagging with naive bayes

In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
bag_number_array = np.arange(2,20,1)
portion_percent_array = np.arange(0.6, 1.3, 0.1)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
bag_number_best = 0
portion_percent_best = 0
accuracy = 0

for bag_number in bag_number_array:
    for portion_percent in portion_percent_array:
        bag_list = naive_bayes_bagging_train('corpus1_train.labels', bag_number, portion_percent)
        naive_bayes_bagging_test('corpus1_test.list', bag_list)
        _, _, accuracy_new = compute_accuracy('corpus1_test.labels','predicted.labels')
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            bag_number_best = bag_number
            portion_percent_best = portion_percent
print("The best parameters for laplace smoothing bagging are bag_number:%d  portion_percent:%1.2f" %(bag_number_best,portion_percent_best))

The best parameters for laplace smoothing bagging are bag_number:2  portion_percent:1.00


# Finding the best parameters for three different smoothing methods

In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
stop_word_array = np.arange(0,30,1)
coef_array = np.logspace(-5,0,30, endpoint=True)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
coef_best = 0
stop_word_best = 0
accuracy = 0

for stop_word in stop_word_array:
    for coef in coef_array:
        accuracy_new = k_fold_cross_validation('corpus2_train.labels',5,stop_word,'lap', coef)
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            coef_best = coef
            stop_word_best = stop_word
print("The best parameters for laplace smoothing are stop_word:%d  coef:%1.5f" %(stop_word_best,coef_best))

The best parameters for laplace smoothing are stop_word:28  coef:0.09237


In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
stop_word_array = np.arange(24,27,1)
coef_array = np.arange(0.09,0.1,0.001)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
coef_best = 0
stop_word_best = 0
accuracy = 0

for stop_word in stop_word_array:
    for coef in coef_array:
        accuracy_new = k_fold_cross_validation('corpus3_train.labels',5,stop_word,'lap', coef)
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            coef_best = coef
            stop_word_best = stop_word
print("The best parameters for laplace smoothing are stop_word:%d  coef:%1.5f" %(stop_word_best,coef_best))

The best parameters for laplace smoothing are stop_word:25  coef:0.09400


In [ ]:
#PRINT ACCURACY WITH THE ABOVE PARAMETERS FOR THREE CORPUS
accuracy_corpus1 = k_fold_cross_validation('corpus1_train.labels',10,28,'lap', 0.09237)
accuracy_corpus2 = k_fold_cross_validation('corpus2_train.labels',10,28,'lap', 0.09237)
accuracy_corpus3 = k_fold_cross_validation('corpus3_train.labels',10,28,'lap', 0.09237)
print("Accuracy for corpus 1, 2, 3 are %1.5f%%, %1.5f%%, %1.5f%% respectively" %(accuracy_corpus1*100, accuracy_corpus2*100, accuracy_corpus3*100))

Accuracy for corpus 1, 2, 3 are 86.32768%, 83.89262%, 91.51832% respectively


In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
stop_word_array = np.arange(10,30,1)
coef_array = np.logspace(-5,0,30, endpoint=True)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
coef_best = 0
stop_word_best = 0
accuracy = 0

for stop_word in stop_word_array:
    for coef in coef_array:
        accuracy_new = k_fold_cross_validation('corpus2_train.labels',5,stop_word,'JM', coef)
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            coef_best = coef
            stop_word_best = stop_word
print("The best parameters for JM smoothing are stop_word:%d  coef:%1.5f" %(stop_word_best,coef_best))

The best parameters for JM smoothing are stop_word:26  coef:0.02807


In [ ]:
#PRINT ACCURACY WITH THE ABOVE PARAMETERS FOR THREE CORPUS
accuracy_corpus1 = k_fold_cross_validation('corpus1_train.labels',10,26,'JM', 0.02807)
accuracy_corpus2 = k_fold_cross_validation('corpus2_train.labels',10,26,'JM', 0.02807)
accuracy_corpus3 = k_fold_cross_validation('corpus3_train.labels',10,26,'JM', 0.02807)
print("Accuracy for corpus 1, 2, 3 are %1.5f%%, %1.5f%%, %1.5f%% respectively" %(accuracy_corpus1*100, accuracy_corpus2*100, accuracy_corpus3*100))

Accuracy for corpus 1, 2, 3 are 86.66667%, 83.44519%, 90.15707% respectively


In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
stop_word_array = np.arange(10,30,1)
coef_array = np.logspace(-5,0,30, endpoint=True)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
coef_best = 0
stop_word_best = 0
accuracy = 0

for stop_word in stop_word_array:
    for coef in coef_array:
        accuracy_new = k_fold_cross_validation('corpus2_train.labels',5,stop_word,'dir', coef)
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            coef_best = coef
            stop_word_best = stop_word
print("The best parameters for Dirichlet smoothing are stop_word:%d  coef:%1.5f" %(stop_word_best,coef_best))

The best parameters for Dirichlet smoothing are stop_word:22  coef:0.02807


In [ ]:
#PRINT ACCURACY WITH THE ABOVE PARAMETERS FOR THREE CORPUS
accuracy_corpus1 = k_fold_cross_validation('corpus1_train.labels',10,22,'dir', 0.02807)
accuracy_corpus2 = k_fold_cross_validation('corpus2_train.labels',10,22,'dir', 0.02807)
accuracy_corpus3 = k_fold_cross_validation('corpus3_train.labels',10,22,'dir', 0.02807)
print("Accuracy for corpus 1, 2, 3 are %1.5f%%, %1.5f%%, %1.5f%% respectively" %(accuracy_corpus1*100, accuracy_corpus2*100, accuracy_corpus3*100))

Accuracy for corpus 1, 2, 3 are 84.06780%, 81.43177%, 86.07330% respectively


In [ ]:
#-----LOOP TO FIND THE BEST PARAMETER SETS
#INITIATE VALUES TO LOOP THROUGH
stop_word_array = np.arange(24,26,1)
coef_array = np.arange(0.09,0.11,0.002)

#INITIATE VARIABLES TO STORE BEST PARAMETERS
coef_best = 0
stop_word_best = 0
accuracy = 0

for stop_word in stop_word_array:
    for coef in coef_array:
        accuracy_new = k_fold_cross_validation('corpus2_train.labels',5,stop_word,'lap', coef)
        if accuracy_new > accuracy:
            accuracy = accuracy_new
            coef_best = coef
            stop_word_best = stop_word
print("The best parameters for laplace smoothing are stop_word:%d  coef:%1.5f" %(stop_word_best,coef_best))

The best parameters for laplace smoothing are stop_word:25  coef:0.10400
